# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

First, make a selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [25]:
# Load necessary packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# remove "object"-type features and SalesPrice from `X`
X = df[[col for col in df.columns if df[col].dtype != np.object and col != 'SalePrice']]
X.head()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


In [26]:
# Impute null values
for col in X.columns:
    X[col].fillna(value=X[col].median(), inplace=True)

# Create y
y = df['SalePrice']

Look at the information of `X` again

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [29]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X,y)
# Fit the model and print R2 and MSE for train and test
lm = LinearRegression()
lm.fit(X_train, y_train)

print('train R2: ', lm.score(X_train, y_train))
print('test R2: ', lm.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, lm.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, lm.predict(X_test)))

train R2:  0.8455258478471324
test R2:  0.6585910197533212
train MSE:  931413971.1871803
test MSE:  2434706069.0402255


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [30]:
from sklearn import preprocessing

# Scale the data and perform train test split
scaled = preprocessing.scale(X)
X_scaled = pd.DataFrame(scaled, columns = X.columns)
X_scaled.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,-1.730865,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,-1.728492,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,-1.726120,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,-1.723747,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,-1.721374,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777


In [31]:
# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)



Perform the same linear regression on this data and print out R-squared and MSE.

In [32]:
# Your code here
lm = LinearRegression()
lm.fit(X_train, y_train)

print('train R2: ', lm.score(X_train, y_train))
print('test R2: ', lm.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, lm.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, lm.predict(X_test)))

train R2:  0.8308217061094032
test R2:  0.7165595583164421
train MSE:  1085978854.279662
test MSE:  1692047515.0820775


## Include dummy variables

Your model hasn't included dummy variables so far: let's use the "object" variables again and create dummies

In [33]:
# Create X_cat which contains only the categorical variables
X_cat = df[[col for col in df.columns if df[col].dtype == np.object]]

In [35]:
# Make dummies
X_cat = pd.get_dummies(X_cat, drop_first=True)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [37]:
# Your code here
X_final = pd.concat([X_scaled, X_cat], axis = 1)
X_final.shape

(1460, 246)

Perform the same linear regression on this data and print out R-squared and MSE.

In [38]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X_final, y)
lm = LinearRegression()
lm.fit(X_train, y_train)

print('train R2: ', lm.score(X_train, y_train))
print('test R2: ', lm.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, lm.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, lm.predict(X_test)))

train R2:  0.9451028181728397
test R2:  -7.337238474274651e+17
train MSE:  326950262.2854261
test MSE:  5.35949062813115e+27


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far off. Similarly, the scale of the Testing MSE is orders of magnitude higher than that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [40]:
# Your code here
from sklearn.linear_model import Lasso, Ridge
lasso = Lasso()
lasso.fit(X_train, y_train)

print('train R2: ', lasso.score(X_train, y_train))
print('test R2: ', lasso.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, lasso.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, lasso.predict(X_test)))

train R2:  0.9449814100827167
test R2:  0.8224839272500283
train MSE:  327673330.4938132
test MSE:  1296667310.9806135


With a higher regularization parameter (alpha = 10)

In [41]:
# Your code here
lasso = Lasso(alpha = 10)
lasso.fit(X_train, y_train)

print('train R2: ', lasso.score(X_train, y_train))
print('test R2: ', lasso.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, lasso.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, lasso.predict(X_test)))

train R2:  0.9402348430143115
test R2:  0.8323257727168677
train MSE:  355942383.591229
test MSE:  1224777486.5896244


## Ridge

With default parameter (alpha = 1)

In [42]:
# Your code here
ridge = Ridge()
ridge.fit(X_train, y_train)

print('train R2: ', ridge.score(X_train, y_train))
print('test R2: ', ridge.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, ridge.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, ridge.predict(X_test)))

train R2:  0.915424290196667
test R2:  0.8620023201937234
train MSE:  503706193.7029152
test MSE:  1008004952.0248127


With default parameter (alpha = 10)

In [43]:
# Your code here
ridge = Ridge(alpha = 10)
ridge.fit(X_train, y_train)

print('train R2: ', ridge.score(X_train, y_train))
print('test R2: ', ridge.score(X_test, y_test))
print('train MSE: ', mean_squared_error(y_train, ridge.predict(X_train)))
print('test MSE: ', mean_squared_error(y_test, ridge.predict(X_test)))

train R2:  0.8938891721670769
test R2:  0.8678876230116731
train MSE:  631962549.562673
test MSE:  965015719.2131671


## Look at the metrics, what are your main conclusions?   

Conclusions here  
test data performed better with Ridge, and performed slightly better with alpha = 10 

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

Compare with the total length of the parameter space and draw conclusions!

In [46]:
# number of Ridge params almost zero

sum(abs(ridge.coef_) < 0.00000000000000001)

5

In [47]:
# number of Lasso params almost zero
sum(abs(lasso.coef_) < 0.00000000000000001)

46

Lasso was very effective to essentially perform variable selection and remove about 25% of the variables from your model!

In [ ]:
# your code here

## Summary

Great! You now know how to perform Lasso and Ridge regression.